In [1]:
import pandas as pd
import os
import ast, json
import math

In [143]:
dff=pd.read_csv(r"E:\KreupAI\DAta\Event\23-24\Almeria vs Real Madrid.csv")
dff.head()

,Unnamed: 0,id,eventId,minute,second,teamId,x,y,expandedMinute,period,...,blockedY,cardType,teamName,oppositionTeamName,shirtNo,name,position,isFirstEleven,ishome,datetime
0,0,2.579155e+09,2,0,0.0,1799,0.0,0.0,0,1,...,NaN,NaN,Almeria,Real Madrid,NaN,NaN,NaN,NaN,1,2023-08-20 17:33:37
1,1,2.579155e+09,2,0,0.0,52,0.0,0.0,0,1,...,NaN,NaN,Real Madrid,Almeria,NaN,NaN,NaN,NaN,0,2023-08-20 17:33:37
2,2,2.579155e+09,3,0,0.0,1799,49.7,49.8,0,1,...,NaN,NaN,Almeria,Real Madrid,10.0,Adri Embarba,AMR,True,1,2023-08-20 17:33:37
3,3,2.579155e+09,4,0,2.0,1799,44.0,45.6,0,1,...,NaN,NaN,Almeria,Real Madrid,5.0,Lucas Robertone,DMC,True,1,2023-08-20 17:33:37
4,4,2.579155e+09,5,0,4.0,1799,60.3,21.9,0,1,...,NaN,NaN,Almeria,Real Madrid,7.0,Largie Ramazani,AML,True,1,2023-08-20 17:33:37


In [147]:
def parse_qualifiers(q):
    # handle NaN/None
    if pd.isna(q):
        return {}

    # if string, try to parse it (handles single-quote Python repr and JSON double-quote)
    if isinstance(q, str):
        s = q.strip()
        if s == "":
            return {}
        parsed = None
        try:
            parsed = ast.literal_eval(s)   # works for Python-style repr using single quotes
        except Exception:
            try:
                parsed = json.loads(s)     # works for JSON-style strings
            except Exception:
                # cannot parse -> assume empty
                return {}
        q = parsed

    # if a single dict, convert to list of one
    if isinstance(q, dict):
        q = [q]

    # if still not a list, bail out
    if not isinstance(q, list):
        return {}

    out = {}
    for item in q:
        if not isinstance(item, dict):
            continue

        # Get displayName
        name = None
        t = item.get('type')
        if isinstance(t, dict):
            name = t.get('displayName')
        # fallback to top-level keys if present
        if not name:
            name = item.get('displayName') or item.get('name')

        if not name:
            continue

        # value: if not present -> 1
        value = item.get('value', 1)
        if value is None or (isinstance(value, str) and value.strip() == ""):
            value = 1

        # If value is list/dict, stringify it
        if isinstance(value, (list, dict)):
            value = str(value)

        # Try convert numeric strings to numbers
        if isinstance(value, str):
            try:
                f = float(value)
                # keep int when appropriate
                if math.isclose(f, int(f)):
                    value = int(f)
                else:
                    value = f
            except Exception:
                # leave as string (eg. 'Back')
                pass

        # keep first occurrence for duplicate displayName
        if name in out:
            continue
        out[name] = value

    return out

# Apply to your dataframe
expanded = dff['qualifiers'].apply(parse_qualifiers)
expanded_dff = pd.json_normalize(expanded)   # each row -> columns
# Optional: convert numeric-like columns to numeric types (keeps non-numeric alone)
def convert_if_number(x):
    try:
        if pd.isna(x):
            return x
        f = float(x)
        if math.isclose(f, int(f)):
            return int(f)
        return f
    except Exception:
        return x

expanded_dff = expanded_dff.applymap(convert_if_number)

# Join back to original df
dff = pd.concat([dff, expanded_dff], axis=1)


C:\Users\kevin\AppData\Local\Temp\ipykernel_23208\4224827824.py:91: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_dff = expanded_dff.applymap(convert_if_number)


In [149]:
columns = [
    # Identifiers & Basic Info
    "id", "eventId", "datetime",

    # Timing
    "minute", "expandedMinute", "second", "period",

    # Teams & Context
    "teamId", "teamName", "oppositionTeamName", "ishome",

    # Location (Ball Position)
    "x", "y", "endX", "endY",

    # Event Details
    "type", "outcomeType", "isTouch", "isShot",

    # Player Information
    "playerId", "name", "shirtNo", "JerseyNumber",
    "position", "PlayerPosition", "isFirstEleven",

    # Match Flags (Goals, Cards, etc.)
    "isGoal", "OwnGoal",
    "Yellow", "VoidYellowCard", "SecondYellow", "Red",

    # Qualifiers & Actions
    "Longball", "Cross", "Foul", "FreekickTaken",
    "ShotAssist", "KeyPass", "IndividualPlay", "RegularPlay",
    "DivingSave", "KeeperSaveObox", "KeeperSaveInTheBox",
    "KeeperThrow", "Blocked", "CornerTaken", "HighClaim","Throughball","LeadingToAttempt",
    "LeadingToGoal","Penalty","PlayerPosition"

    "IntentionalAssist", "IntentionalGoalAssist", "GoalKick",
    "FromCorner", "BigChanceCreated", "BigChance", "FastBreak",
    "LeadingToAttempt", "OneOnOne", "SetPiece",
    "KeeperSaved", "KeeperSaveInSixYard","OutfielderBlock"
]
for col in columns:
    if col not in dff.columns:
        dff[col] = 0

# Reorder/select only the required columns
dff = dff[columns]

In [153]:
dff['type'].unique()

array(['Start', 'Pass', 'BallRecovery', 'BallTouch', 'BlockedPass',
       'Interception', 'Dispossessed', 'Tackle', 'TakeOn', 'Foul',
       'KeeperPickup', 'Goal', 'SavedShot', 'Save', 'CornerAwarded',
       'Aerial', 'MissedShots', 'Clearance', 'Challenge', 'OffsideGiven',
       'OffsidePass', 'OffsideProvoked', 'Claim', 'End', 'Card',
       'CrossNotClaimed', 'Error', 'SubstitutionOff', 'SubstitutionOn',
       'FormationChange', 'ShieldBallOpp', 'FormationSet'], dtype=object)

In [181]:


full_time = 90

# Unique player entries (team, position info)
player_keys = dff[['name', 'teamId', 'teamName', 'shirtNo', 'position','playerId','ishome','datetime']].drop_duplicates() #'PlayerPosition',
player_keys = player_keys.dropna(subset=['name']).reset_index(drop=True)

rows = []
for _, r in player_keys.iterrows():
    name, team_id, team_name, shirt, position,playerid,ishome,timestamp = (
        r['name'], r['teamId'], r['teamName'], r['shirtNo'], r['position'],r['playerId'],r['ishome'],r['datetime']
    )
    pe = dff[
        (dff['name'] == name)
        & (dff['teamId'] == team_id)
        & (dff['shirtNo'] == shirt)
    ]
    touches=(pe['isTouch'] == True).sum()
    shots=(pe['isShot'] == True).sum()
    goals=((pe['type'] == 'Goal')& (pe['OwnGoal'] !=1)).sum() 
    pengoals=((pe['type'] == 'Goal')& (pe['Penalty'] ==1)).sum()
    missedpens=((pe['type'] != 'Goal')& (pe['Penalty'] ==1)).sum()
    # === Passing stats ===
    assists=(pe['IntentionalGoalAssist'] == 1).sum()
    keypass=(pe['KeyPass'] == 1).sum()
    
    bcc=(pe['BigChanceCreated'] == 1).sum()
    disp = (pe['type'] == 'Dispossessed').sum()
    
    passatt = (pe['type'] == 'Pass').sum()
    succpass = ((pe['type'] == 'Pass') & (pe['outcomeType'] == 'Successful')).sum()
    
    dribbles = (pe['type'] == 'TakeOn').sum()
    succdrib = ((pe['type'] == 'TakeOn') & (pe['outcomeType'] == 'Successful')).sum()
    
    errors = (pe['type'] == 'Error').sum()
    # === Cross stats ===
    crossesatt = ((pe['Cross'] == 1)&(pe['type'] == 'Pass')).sum()
    succcross = ((pe['type'] == 'Pass') &(pe['Cross'] == 1) & (pe['outcomeType'] == 'Successful')).sum()
    
    # === Long ball stats ===
    longballatt = ((pe['Longball'] == 1)&(pe['type'] == 'Pass')).sum()
    succlongball = ((pe['type'] == 'Pass')&(pe['Longball'] == 1) & (pe['outcomeType'] == 'Successful')).sum()
    throughballatt = ((pe['Throughball'] == 1)&(pe['type'] == 'Pass')).sum()
    succthroughball = ((pe['type'] == 'Pass')&(pe['Throughball'] == 1) & (pe['outcomeType'] == 'Successful')).sum()
    
    tackles = (pe['type'] == 'Tackle').sum()
    succtackles = ((pe['type'] == 'Tackle')& (pe['outcomeType'] == 'Unsuccessful')).sum()
    interceptions = (pe['type'] == 'Interception').sum()
    clearances = (pe['type'] == 'Clearance').sum()
    fouls = ((pe['type'] == 'Foul')& (pe['outcomeType'] == 'Unsuccessful')).sum()
    fouled = ((pe['type'] == 'Foul')& (pe['outcomeType'] == 'Successful')).sum()
    blockedshots = ((pe['type'] == 'Save')& (pe['OutfielderBlock'] == 1)).sum()
    
    # Substitution info
    subs_on_vals = pe.loc[pe['type'] == 'SubstitutionOn', 'minute']
    subs_off_vals = pe.loc[pe['type'] == 'SubstitutionOff', 'minute']
    subs_on_min = subs_on_vals.min() if not subs_on_vals.empty else None
    subs_off_min = subs_off_vals.min() if not subs_off_vals.empty else None

    # === Minutes played calculation ===
    if pd.notna(subs_on_min):
        start = int(subs_on_min)
        if start >= full_time:
            mins = 1
        else:
            end = int(subs_off_min) if pd.notna(subs_off_min) else full_time
            end = min(end, full_time)
            mins = end - start
            if mins < 1:
                mins = 1
    elif pe['isFirstEleven'].any():
        end = int(subs_off_min) if pd.notna(subs_off_min) else full_time
        end = min(end, full_time)
        mins = end
        if mins < 0:
            mins = 0
    else:
        mins = 0

    rows.append({
        'playerId':int(playerid),
        'name': name,
        'teamId': int(team_id),
        'teamName': team_name,
        'ishome':ishome,
        'shirtNo': int(shirt),
        'position': position,
       'timestamp':timestamp,
        'minutes_played': int(mins),
        'shots':shots,
        'goals':goals,
        'assists':assists,
        'keypass':keypass,
        'bcc':bcc,
        'Touches':touches,
        'dribbles':dribbles,
        'disp':disp,
        'succdrib':succdrib,
        'Passatt': int(passatt),
        'succPass': int(succpass),
        'Crossesatt': int(crossesatt),
        'succCross': int(succcross),
        'Longballatt': int(longballatt),
        'succLongball': int(succlongball),
        'Throughballatt':int(throughballatt),
        'succThroughball':int(succthroughball),
        'TotTackles':int(tackles),
        'succTackles':int(succtackles),
        'Interceptions':int(interceptions),
        'Clearances':clearances,
        'blockedshots':blockedshots,
        'Fouls':fouls,
        'Fouled':fouled
    })

stats_df = (
    pd.DataFrame(rows)
    .sort_values(['teamName', 'minutes_played'], ascending=[True, False])
    .reset_index(drop=True)
)
stats_df.to_csv("testing.csv")
stats_df


,playerId,name,teamId,teamName,ishome,shirtNo,position,timestamp,minutes_played,shots,...,succLongball,Throughballatt,succThroughball,TotTackles,succTackles,Interceptions,Clearances,blockedshots,Fouls,Fouled
0,135572,Adri Embarba,1799,Almeria,1,10,AMR,2023-08-20 17:33:37,90,0,...,1,0,0,4,3,0,0,0,0,1
1,387160,Edgar Gonzalez,1799,Almeria,1,3,DC,2023-08-20 17:33:37,90,1,...,5,0,0,3,1,1,9,1,1,0
2,365127,Chumi,1799,Almeria,1,21,DC,2023-08-20 17:33:37,90,0,...,0,0,0,1,1,0,5,2,3,0
3,402887,Luis Suarez,1799,Almeria,1,9,FW,2023-08-20 17:33:37,90,5,...,1,0,0,0,0,0,0,0,1,1
4,401174,Sergio Arribas,1799,Almeria,1,19,AMC,2023-08-20 17:33:37,90,1,...,3,0,0,0,0,0,0,0,1,2
5,361969,Luis Maximiano,1799,Almeria,1,25,GK,2023-08-20 17:33:37,90,0,...,8,0,0,0,0,0,0,0,0,0
6,301773,Sergio Akieme,1799,Almeria,1,15,DL,2023-08-20 17:33:37,90,0,...,6,0,0,0,0,0,2,0,0,2
7,381644,Iddrisu Baba,1799,Almeria,1,4,DMC,2023-08-20 17:33:37,90,1,...,3,0,0,3,2,2,1,0,0,2
8,323511,Lucas Robertone,1799,Almeria,1,5,DMC,2023-08-20 17:33:37,68,5,...,3,1,0,0,0,2,1,1,3,1
9,430148,Marc Pubill,1799,Almeria,1,18,DR,2023-08-20 17:33:37,68,0,...,2,0,0,3,0,2,2,0,1,3


In [183]:
df=pd.read_csv(r"E:\KreupAI\DAta\PlayerStat\23-24\Almeria vs Real Madrid.csv")
df

,Unnamed: 0,incidents,playedPositionsShort,height,weight,age,isManOfTheMatch,isActive,playedPositions,teamRegionId,...,turnover,dispossessed,duelAerialWon,totalAccuratePasses,totalPasses,passSuccess,ranking,teamname,ishome,datetime
0,0,"[{'id': 2579237165.0, 'eventId': 499, 'minute'...",GK,0,0,26,False,True,NaN,0,...,0.0,0.0,0.0,0.0,29.0,0.0,0,Almeria,1,2023-08-20 17:33:37
1,1,"[{'id': 2579245725.0, 'eventId': 563, 'minute'...",DR,0,0,22,False,True,NaN,0,...,0.0,1.0,0.0,0.0,31.0,0.0,0,Almeria,1,2023-08-20 17:33:37
2,2,[],DC,0,0,28,False,True,NaN,0,...,0.0,1.0,1.0,0.0,47.0,0.0,0,Almeria,1,2023-08-20 17:33:37
3,3,"[{'id': 2579236533.0, 'eventId': 497, 'minute'...",DC,0,0,26,False,True,NaN,0,...,1.0,0.0,0.0,0.0,33.0,0.0,0,Almeria,1,2023-08-20 17:33:37
4,4,[],DL,0,0,27,False,True,NaN,0,...,0.0,0.0,0.0,0.0,42.0,0.0,0,Almeria,1,2023-08-20 17:33:37
5,5,"[{'id': 2579157213.0, 'eventId': 22, 'minute':...",DMC,0,0,28,False,True,NaN,0,...,1.0,0.0,1.0,0.0,39.0,0.0,0,Almeria,1,2023-08-20 17:33:37
6,6,"[{'id': 2579269847.0, 'eventId': 728, 'minute'...",DMC,0,0,29,False,True,NaN,0,...,0.0,0.0,2.0,0.0,46.0,0.0,0,Almeria,1,2023-08-20 17:33:37
7,7,[],AMR,0,0,33,False,True,NaN,0,...,3.0,0.0,3.0,0.0,47.0,0.0,0,Almeria,1,2023-08-20 17:33:37
8,8,"[{'id': 2579157215.0, 'eventId': 23, 'minute':...",AMC,0,0,23,False,True,NaN,0,...,0.0,1.0,0.0,0.0,22.0,0.0,0,Almeria,1,2023-08-20 17:33:37
9,9,"[{'id': 2579238713.0, 'eventId': 510, 'minute'...",AML,0,0,24,False,True,NaN,0,...,1.0,1.0,0.0,0.0,13.0,0.0,0,Almeria,1,2023-08-20 17:33:37


In [185]:
import pandas as pd
import os
import ast, json
import math

# === CONFIGURATION ===
events_dir = r"E:\KreupAI\DAta\Event\23-24"
stats_dir = r"E:\KreupAI\DAta\PlayerStat\23-24.csv"  # PlayerStat folder or CSV files
output_dir = r"E:\KreupAI\DAta\ownplayerstat"
full_time = 90

os.makedirs(output_dir, exist_ok=True)

# --- Helper to parse qualifiers ---
def parse_qualifiers(q):
    if pd.isna(q):
        return {}
    if isinstance(q, str):
        s = q.strip()
        if s == "":
            return {}
        try:
            q = ast.literal_eval(s)
        except Exception:
            try:
                q = json.loads(s)
            except Exception:
                return {}
    if isinstance(q, dict):
        q = [q]
    if not isinstance(q, list):
        return {}

    out = {}
    for item in q:
        if not isinstance(item, dict):
            continue
        name = None
        t = item.get('type')
        if isinstance(t, dict):
            name = t.get('displayName')
        if not name:
            name = item.get('displayName') or item.get('name')
        if not name:
            continue

        value = item.get('value', 1)
        if value is None or (isinstance(value, str) and value.strip() == ""):
            value = 1
        if isinstance(value, (list, dict)):
            value = str(value)
        if isinstance(value, str):
            try:
                f = float(value)
                value = int(f) if math.isclose(f, int(f)) else f
            except Exception:
                pass
        if name not in out:
            out[name] = value
    return out

# --- Process each match file ---
for file in os.listdir(events_dir):
    if not file.endswith(".csv"):
        continue

    match_name = os.path.splitext(file)[0]
    event_path = os.path.join(events_dir, file)
    playerstat_path = os.path.join(r"E:\KreupAI\DAta\PlayerStat", file)

    if not os.path.exists(playerstat_path):
        print(f"Skipping {match_name}: corresponding PlayerStat file not found.")
        continue

    # === Read Event Data ===
    dff = pd.read_csv(event_path)
    expanded = dff['qualifiers'].apply(parse_qualifiers)
    expanded_dff = pd.json_normalize(expanded)

    def convert_if_number(x):
        try:
            if pd.isna(x):
                return x
            f = float(x)
            return int(f) if math.isclose(f, int(f)) else f
        except Exception:
            return x

    expanded_dff = expanded_dff.applymap(convert_if_number)
    dff = pd.concat([dff, expanded_dff], axis=1)

    # === Ensure required columns ===
    columns = [
        "id","eventId","datetime","minute","expandedMinute","second","period",
        "teamId","teamName","oppositionTeamName","ishome","x","y","endX","endY",
        "type","outcomeType","isTouch","isShot","playerId","name","shirtNo","JerseyNumber",
        "position","PlayerPosition","isFirstEleven","isGoal","OwnGoal","Yellow","VoidYellowCard",
        "SecondYellow","Red","Longball","Cross","Foul","FreekickTaken","ShotAssist","KeyPass",
        "IndividualPlay","RegularPlay","DivingSave","KeeperSaveObox","KeeperSaveInTheBox",
        "KeeperThrow","Blocked","CornerTaken","HighClaim","Throughball","LeadingToAttempt",
        "LeadingToGoal","Penalty","PlayerPosition","IntentionalAssist","IntentionalGoalAssist",
        "GoalKick","FromCorner","BigChanceCreated","BigChance","FastBreak","LeadingToAttempt",
        "OneOnOne","SetPiece","KeeperSaved","KeeperSaveInSixYard","OutfielderBlock"
    ]
    for col in columns:
        if col not in dff.columns:
            dff[col] = 0
    dff = dff[columns]

    # === Build stats_df ===
    player_keys = dff[['name','teamId','teamName','shirtNo','position','playerId','ishome','datetime']].drop_duplicates()
    player_keys = player_keys.dropna(subset=['name']).reset_index(drop=True)

    rows = []
    for _, r in player_keys.iterrows():
        name, team_id, team_name, shirt, position, pid, ishome, timestamp = (
            r['name'], r['teamId'], r['teamName'], r['shirtNo'], r['position'], r['playerId'], r['ishome'], r['datetime']
        )
        pe = dff[(dff['name']==name)&(dff['teamId']==team_id)&(dff['shirtNo']==shirt)]
        touches = (pe['isTouch']==True).sum()
        shots = (pe['isShot']==True).sum()
        goals = ((pe['type']=='Goal')&(pe['OwnGoal']!=1)).sum()
        pengoals = ((pe['type']=='Goal')&(pe['Penalty']==1)).sum()
        missedpens = ((pe['type']!='Goal')&(pe['Penalty']==1)).sum()
        assists = (pe['IntentionalGoalAssist']==1).sum()
        keypass = (pe['KeyPass']==1).sum()
        bcc = (pe['BigChanceCreated']==1).sum()
        disp = (pe['type']=='Dispossessed').sum()
        passatt = (pe['type']=='Pass').sum()
        succpass = ((pe['type']=='Pass')&(pe['outcomeType']=='Successful')).sum()
        dribbles = (pe['type']=='TakeOn').sum()
        succdrib = ((pe['type']=='TakeOn')&(pe['outcomeType']=='Successful')).sum()
        errors = (pe['type']=='Error').sum()
        crossesatt = ((pe['Cross']==1)&(pe['type']=='Pass')).sum()
        succcross = ((pe['type']=='Pass')&(pe['Cross']==1)&(pe['outcomeType']=='Successful')).sum()
        longballatt = ((pe['Longball']==1)&(pe['type']=='Pass')).sum()
        succlongball = ((pe['type']=='Pass')&(pe['Longball']==1)&(pe['outcomeType']=='Successful')).sum()
        throughballatt = ((pe['Throughball']==1)&(pe['type']=='Pass')).sum()
        succthroughball = ((pe['type']=='Pass')&(pe['Throughball']==1)&(pe['outcomeType']=='Successful')).sum()
        tackles = (pe['type']=='Tackle').sum()
        succtackles = ((pe['type']=='Tackle')&(pe['outcomeType']=='Unsuccessful')).sum()
        interceptions = (pe['type']=='Interception').sum()
        clearances = (pe['type']=='Clearance').sum()
        fouls = ((pe['type']=='Foul')&(pe['outcomeType']=='Unsuccessful')).sum()
        fouled = ((pe['type']=='Foul')&(pe['outcomeType']=='Successful')).sum()
        blockedshots = ((pe['type']=='Save')&(pe['OutfielderBlock']==1)).sum()

        subs_on_vals = pe.loc[pe['type']=='SubstitutionOn','minute']
        subs_off_vals = pe.loc[pe['type']=='SubstitutionOff','minute']
        subs_on_min = subs_on_vals.min() if not subs_on_vals.empty else None
        subs_off_min = subs_off_vals.min() if not subs_off_vals.empty else None

        if pd.notna(subs_on_min):
            start = int(subs_on_min)
            if start >= full_time:
                mins = 1
            else:
                end = int(subs_off_min) if pd.notna(subs_off_min) else full_time
                end = min(end, full_time)
                mins = end - start
                if mins < 1:
                    mins = 1
        elif pe['isFirstEleven'].any():
            end = int(subs_off_min) if pd.notna(subs_off_min) else full_time
            end = min(end, full_time)
            mins = end
            if mins < 0:
                mins = 0
        else:
            mins = 0

        rows.append({
            'playerId': int(pid),
            'name': name,
            'teamId': int(team_id),
            'teamName': team_name,
            'ishome': ishome,
            'shirtNo': int(shirt),
            'position': position,
            'timestamp': timestamp,
            'minutes_played': int(mins),
            'shots': shots,
            'goals': goals,
            'assists': assists,
            'keypass': keypass,
            'bcc': bcc,
            'Touches': touches,
            'dribbles': dribbles,
            'disp': disp,
            'succdrib': succdrib,
            'Passatt': int(passatt),
            'succPass': int(succpass),
            'Crossesatt': int(crossesatt),
            'succCross': int(succcross),
            'Longballatt': int(longballatt),
            'succLongball': int(succlongball),
            'Throughballatt': int(throughballatt),
            'succThroughball': int(succthroughball),
            'TotTackles': int(tackles),
            'succTackles': int(succtackles),
            'Interceptions': int(interceptions),
            'Clearances': clearances,
            'blockedshots': blockedshots,
            'Fouls': fouls,
            'Fouled': fouled
        })

    stats_df = pd.DataFrame(rows).sort_values(['teamName','minutes_played'], ascending=[True,False]).reset_index(drop=True)

    # === Merge with PlayerStat ===
    df_stats = pd.read_csv(playerstat_path)
    merged_df = pd.merge(stats_df, df_stats[['playerId','isManOfTheMatch','age','rating']], on='playerId', how='left')
    merged_df['isManOfTheMatch'] = merged_df['isManOfTheMatch'].apply(lambda x: 1 if x is True else 0)

    # === Save Output ===
    output_file = os.path.join(output_dir, f"{match_name}_ownplayerstat.csv")
    merged_df.to_csv(output_file, index=False)
    print(f"Saved: {output_file}")


In [187]:
merged_df

,playerId,name,teamId,teamName,ishome,shirtNo,position,timestamp,minutes_played,shots,...,TotTackles,succTackles,Interceptions,Clearances,blockedshots,Fouls,Fouled,isManOfTheMatch,age,rating
0,135572,Adri Embarba,1799,Almeria,1,10,AMR,2023-08-20 17:33:37,90,0,...,4,3,0,0,0,0,1,0,33,7.59
1,387160,Edgar Gonzalez,1799,Almeria,1,3,DC,2023-08-20 17:33:37,90,1,...,3,1,1,9,1,1,0,0,28,6.64
2,365127,Chumi,1799,Almeria,1,21,DC,2023-08-20 17:33:37,90,0,...,1,1,0,5,2,3,0,0,26,5.78
3,402887,Luis Suarez,1799,Almeria,1,9,FW,2023-08-20 17:33:37,90,5,...,0,0,0,0,0,1,1,0,27,6.34
4,401174,Sergio Arribas,1799,Almeria,1,19,AMC,2023-08-20 17:33:37,90,1,...,0,0,0,0,0,1,2,0,23,7.12
5,361969,Luis Maximiano,1799,Almeria,1,25,GK,2023-08-20 17:33:37,90,0,...,0,0,0,0,0,0,0,0,26,5.80
6,301773,Sergio Akieme,1799,Almeria,1,15,DL,2023-08-20 17:33:37,90,0,...,0,0,0,2,0,0,2,0,27,5.91
7,381644,Iddrisu Baba,1799,Almeria,1,4,DMC,2023-08-20 17:33:37,90,1,...,3,2,2,1,0,0,2,0,29,6.91
8,323511,Lucas Robertone,1799,Almeria,1,5,DMC,2023-08-20 17:33:37,68,5,...,0,0,2,1,1,3,1,0,28,7.49
9,430148,Marc Pubill,1799,Almeria,1,18,DR,2023-08-20 17:33:37,68,0,...,3,0,2,2,0,1,3,0,22,6.85


In [201]:
import pandas as pd
import os
import ast, json
import math

# === CONFIGURATION ===
events_dir = r"E:\KreupAI\DAta\Event\24-25"
stats_dir = r"E:\KreupAI\DAta\PlayerStat\24-25"  # PlayerStat folder or CSV files
output_dir = r"E:\KreupAI\DAta\ownplayerstat"
full_time = 90

os.makedirs(output_dir, exist_ok=True)

# --- Helper to parse qualifiers ---
def parse_qualifiers(q):
    if pd.isna(q):
        return {}
    if isinstance(q, str):
        s = q.strip()
        if s == "":
            return {}
        try:
            q = ast.literal_eval(s)
        except Exception:
            try:
                q = json.loads(s)
            except Exception:
                return {}
    if isinstance(q, dict):
        q = [q]
    if not isinstance(q, list):
        return {}

    out = {}
    for item in q:
        if not isinstance(item, dict):
            continue
        name = None
        t = item.get('type')
        if isinstance(t, dict):
            name = t.get('displayName')
        if not name:
            name = item.get('displayName') or item.get('name')
        if not name:
            continue

        value = item.get('value', 1)
        if value is None or (isinstance(value, str) and value.strip() == ""):
            value = 1
        if isinstance(value, (list, dict)):
            value = str(value)
        if isinstance(value, str):
            try:
                f = float(value)
                value = int(f) if math.isclose(f, int(f)) else f
            except Exception:
                pass
        if name not in out:
            out[name] = value
    return out

# --- Process each match file ---
for file in os.listdir(events_dir):
    if not file.endswith(".csv"):
        continue

    match_name = os.path.splitext(file)[0]
    event_path = os.path.join(events_dir, file)
    playerstat_path = os.path.join(r"E:\KreupAI\DAta\PlayerStat\24-25", file)

    if not os.path.exists(playerstat_path):
        print(f"Skipping {match_name}: corresponding PlayerStat file not found.")
        continue

    # === Read Event Data ===
    dff = pd.read_csv(event_path)
    expanded = dff['qualifiers'].apply(parse_qualifiers)
    expanded_dff = pd.json_normalize(expanded)

    def convert_if_number(x):
        try:
            if pd.isna(x):
                return x
            f = float(x)
            return int(f) if math.isclose(f, int(f)) else f
        except Exception:
            return x

    expanded_dff = expanded_dff.applymap(convert_if_number)
    dff = pd.concat([dff, expanded_dff], axis=1)

    # === Ensure required columns ===
    columns = [
        "id","eventId","datetime","minute","expandedMinute","second","period",
        "teamId","teamName","oppositionTeamName","ishome","x","y","endX","endY",
        "type","outcomeType","isTouch","isShot","playerId","name","shirtNo","JerseyNumber",
        "position","PlayerPosition","isFirstEleven","isGoal","OwnGoal","Yellow","VoidYellowCard",
        "SecondYellow","Red","Longball","Cross","Foul","FreekickTaken","ShotAssist","KeyPass",
        "IndividualPlay","RegularPlay","DivingSave","KeeperSaveObox","KeeperSaveInTheBox",
        "KeeperThrow","Blocked","CornerTaken","HighClaim","Throughball","LeadingToAttempt",
        "LeadingToGoal","Penalty","PlayerPosition","IntentionalAssist","IntentionalGoalAssist",
        "GoalKick","FromCorner","BigChanceCreated","BigChance","FastBreak","LeadingToAttempt",
        "OneOnOne","SetPiece","KeeperSaved","KeeperSaveInSixYard","OutfielderBlock"
    ]
    for col in columns:
        if col not in dff.columns:
            dff[col] = 0
    dff = dff[columns]

    # === Build stats_df ===
    player_keys = dff[['name','teamId','teamName','shirtNo','position','playerId','ishome','datetime']].drop_duplicates()
    player_keys = player_keys.dropna(subset=['name']).reset_index(drop=True)

    rows = []
    for _, r in player_keys.iterrows():
        name, team_id, team_name, shirt, position, pid, ishome, timestamp = (
            r['name'], r['teamId'], r['teamName'], r['shirtNo'], r['position'], r['playerId'], r['ishome'], r['datetime']
        )
        pe = dff[(dff['name']==name)&(dff['teamId']==team_id)&(dff['shirtNo']==shirt)]
        touches = (pe['isTouch']==True).sum()
        shots = (pe['isShot']==True).sum()
        goals = ((pe['type']=='Goal')&(pe['OwnGoal']!=1)).sum()
        pengoals = ((pe['type']=='Goal')&(pe['Penalty']==1)).sum()
        missedpens = ((pe['type']!='Goal')&(pe['Penalty']==1)).sum()
        assists = (pe['IntentionalGoalAssist']==1).sum()
        keypass = (pe['KeyPass']==1).sum()
        bcc = (pe['BigChanceCreated']==1).sum()
        disp = (pe['type']=='Dispossessed').sum()
        passatt = (pe['type']=='Pass').sum()
        succpass = ((pe['type']=='Pass')&(pe['outcomeType']=='Successful')).sum()
        dribbles = (pe['type']=='TakeOn').sum()
        succdrib = ((pe['type']=='TakeOn')&(pe['outcomeType']=='Successful')).sum()
        errors = (pe['type']=='Error').sum()
        crossesatt = ((pe['Cross']==1)&(pe['type']=='Pass')).sum()
        succcross = ((pe['type']=='Pass')&(pe['Cross']==1)&(pe['outcomeType']=='Successful')).sum()
        longballatt = ((pe['Longball']==1)&(pe['type']=='Pass')).sum()
        succlongball = ((pe['type']=='Pass')&(pe['Longball']==1)&(pe['outcomeType']=='Successful')).sum()
        throughballatt = ((pe['Throughball']==1)&(pe['type']=='Pass')).sum()
        succthroughball = ((pe['type']=='Pass')&(pe['Throughball']==1)&(pe['outcomeType']=='Successful')).sum()
        tackles = (pe['type']=='Tackle').sum()
        succtackles = ((pe['type']=='Tackle')&(pe['outcomeType']=='Unsuccessful')).sum()
        interceptions = (pe['type']=='Interception').sum()
        clearances = (pe['type']=='Clearance').sum()
        fouls = ((pe['type']=='Foul')&(pe['outcomeType']=='Unsuccessful')).sum()
        fouled = ((pe['type']=='Foul')&(pe['outcomeType']=='Successful')).sum()
        blockedshots = ((pe['type']=='Save')&(pe['OutfielderBlock']==1)).sum()

        subs_on_vals = pe.loc[pe['type']=='SubstitutionOn','minute']
        subs_off_vals = pe.loc[pe['type']=='SubstitutionOff','minute']
        subs_on_min = subs_on_vals.min() if not subs_on_vals.empty else None
        subs_off_min = subs_off_vals.min() if not subs_off_vals.empty else None

        if pd.notna(subs_on_min):
            start = int(subs_on_min)
            if start >= full_time:
                mins = 1
            else:
                end = int(subs_off_min) if pd.notna(subs_off_min) else full_time
                end = min(end, full_time)
                mins = end - start
                if mins < 1:
                    mins = 1
        elif pe['isFirstEleven'].any():
            end = int(subs_off_min) if pd.notna(subs_off_min) else full_time
            end = min(end, full_time)
            mins = end
            if mins < 0:
                mins = 0
        else:
            mins = 0

        rows.append({
            'playerId': int(pid),
            'name': name,
            'teamId': int(team_id),
            'teamName': team_name,
            'ishome': ishome,
            'shirtNo': int(shirt),
            'position': position,
            'timestamp': timestamp,
            'minutes_played': int(mins),
            'shots': shots,
            'goals': goals,
            'assists': assists,
            'keypass': keypass,
            'bcc': bcc,
            'Touches': touches,
            'dribbles': dribbles,
            'disp': disp,
            'succdrib': succdrib,
            'Passatt': int(passatt),
            'succPass': int(succpass),
            'Crossesatt': int(crossesatt),
            'succCross': int(succcross),
            'Longballatt': int(longballatt),
            'succLongball': int(succlongball),
            'Throughballatt': int(throughballatt),
            'succThroughball': int(succthroughball),
            'TotTackles': int(tackles),
            'succTackles': int(succtackles),
            'Interceptions': int(interceptions),
            'Clearances': clearances,
            'blockedshots': blockedshots,
            'Fouls': fouls,
            'Fouled': fouled
        })

    stats_df = pd.DataFrame(rows).sort_values(['teamName','minutes_played'], ascending=[True,False]).reset_index(drop=True)

    # === Merge with PlayerStat ===
    df_stats = pd.read_csv(playerstat_path)
    merged_df = pd.merge(stats_df, df_stats[['playerId','isManOfTheMatch','age','rating']], on='playerId', how='left')
    merged_df['isManOfTheMatch'] = merged_df['isManOfTheMatch'].apply(lambda x: 1 if x is True else 0)
    merged_df['matchname']=match_name
    merged_df['season']='2024-25'

    # === Save Output ===
    output_file = os.path.join(output_dir, f"{match_name}.csv")
    merged_df.to_csv(output_file, index=False)
    print(f"Saved: {output_file}")


C:\Users\kevin\AppData\Local\Temp\ipykernel_23208\70385047.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_dff = expanded_dff.applymap(convert_if_number)


Saved: E:\KreupAI\DAta\ownplayerstat\Athletic Club vs Atletico.csv


C:\Users\kevin\AppData\Local\Temp\ipykernel_23208\70385047.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_dff = expanded_dff.applymap(convert_if_number)


Saved: E:\KreupAI\DAta\ownplayerstat\Athletic Club vs Barcelona.csv


C:\Users\kevin\AppData\Local\Temp\ipykernel_23208\70385047.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_dff = expanded_dff.applymap(convert_if_number)


...

Saved: E:\KreupAI\DAta\ownplayerstat\Villarreal vs Real Valladolid.csv


C:\Users\kevin\AppData\Local\Temp\ipykernel_23208\70385047.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_dff = expanded_dff.applymap(convert_if_number)


Saved: E:\KreupAI\DAta\ownplayerstat\Villarreal vs Sevilla.csv


C:\Users\kevin\AppData\Local\Temp\ipykernel_23208\70385047.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  expanded_dff = expanded_dff.applymap(convert_if_number)


Saved: E:\KreupAI\DAta\ownplayerstat\Villarreal vs Valencia.csv
